### Introduction

In this challenge, we aim to predict the active power signal of a wind turbine using 10-minute SCADA data from neighbouring turbines. The code for all my submissions can be found [here](https://github.com/davidbossanyi/hill-of-towie-wind-turbine-power-prediction).

In [ ]:
# Comment these lines out when developing locally
! pip install ephem 'flaml[automl]'
%cd /kaggle/working

# set the time budget per fit (recommend 180s locally, 240s on Kaggle)
time_budget = 240

In [ ]:
import datetime as dt
import math
from pathlib import Path

import ephem
import kagglehub
import matplotlib.pyplot as plt
import polars as pl
import requests
from flaml.automl import AutoML
from kagglehub.config import DEFAULT_CACHE_FOLDER

In [ ]:
class SunPosition:
    def __init__(self, *, latitude: float, longitude: float) -> None:
        self.latitude = latitude
        self.longitude = longitude
        self._observer = self._create_ephem_observer()
        self._sun = ephem.Sun()

    def _create_ephem_observer(self) -> ephem.Observer:
        observer = ephem.Observer()
        observer.lat = str(self.latitude)
        observer.lon = str(self.longitude)
        return observer

    def altitude(self, *, timestamp_utc: dt.datetime) -> float:
        self._observer.date = timestamp_utc
        self._sun.compute(self._observer)
        return self._sun.alt

In [ ]:
CACHE_DIR = Path(DEFAULT_CACHE_FOLDER) / "competitions" / "hill-of-towie-wind-turbine-power-prediction"


def load_training_dataset(*, force_download: bool = False) -> pl.LazyFrame:
    file_path = kagglehub.competition_download(
        handle="hill-of-towie-wind-turbine-power-prediction",
        path="training_dataset.parquet",
        force_download=force_download,
    )
    return pl.scan_parquet(Path(file_path))


def load_submission_dataset(*, force_download: bool = False) -> pl.LazyFrame:
    file_path = kagglehub.competition_download(
        handle="hill-of-towie-wind-turbine-power-prediction",
        path="submission_dataset.parquet",
        force_download=force_download,
    )
    return pl.scan_parquet(Path(file_path))


def load_turbine_metadata(*, force_download: bool = False) -> pl.LazyFrame:
    file_path = CACHE_DIR / "turbine_metadata.csv"
    if not file_path.exists() or force_download:
        response = requests.get(
            "https://zenodo.org/records/14870023/files/Hill_of_Towie_turbine_metadata.csv?download=1",
            headers={"Accept": "text/csv"},
            timeout=10,
        )
        response.raise_for_status()
        file_path.parent.mkdir(parents=True, exist_ok=True)
        file_path.write_text(response.content.decode("utf-8-sig"), encoding="utf-8")
    return pl.scan_csv(file_path)

### Model

#### Load the training dataset

In [ ]:
df_train = load_training_dataset().collect()
df_train.head(5)

#### Basic feature engineering and selection
I have deliberately chosen to keep only features derived from the wind turbine SCADA data, since other sources such as ERA5 may not be accessible in real time at a real wind farm.

The data is preprocessed and filtered as follows:
 - Create an "age of data" feature that counts the number of seconds since 2016
 - Choose yaw position over nacelle position (same information) as a proxy for local wind direction and calculate its `sin` and `cos` since it is a circular variable, and the data spans more than 360 degrees
 - Take the mean of the reference wind turbine ambient temperatures as a single feature, since the data is very similar across turbines
 - Include the 10-minute lagged wind speed and active power as features, since the partial autocorrelation is fairly strong with a lag of 10 minutes
 - Include sinusoidal time features (hour of day, day of year, month of year) to capture daily and seasonal patterns
 - Include the altitude of the sun as a feature, as a very rough proxy for atmospheric stability
 - Filter on the `is_valid` column - we can't hope to predict turbine faults or missing data, and in any case, the Kaggle scoring excludes invalid rows

We retain only the following features:
 - wind speed (mean, min, max, std, 10-min lag of mean) for all reference turbines
 - active power (mean, min, max, std, 10-min lag of mean) for all reference turbines
 - sin and cos of mean yaw position for all reference turbines, and the yaw position std
 - time in operation for all reference turbines (the model needs to know which turbines are offline so it can learn waking scenarios)
 - generator rpm for all reference turbines
 - pitch position (just blade A to reduce feature redundancy) for all reference turbines
 - mean ambient temperature, as described above
 - sin and cos of time features, as described above
 - altitude of the sun (as described above)
 - age of data (as described above)

In [ ]:
def preprocess(X: pl.DataFrame, *, ref_wtgs: list[int], lat: float, lon: float) -> pl.DataFrame:
    sun_position = SunPosition(latitude=lat, longitude=lon)
    X = (
        X.lazy()
        .with_columns(
            pl.col("TimeStamp_StartFormat")
            .sub(dt.datetime(2016, 1, 1, tzinfo=dt.UTC))
            .dt.total_seconds()
            .alias("seconds_since_2016"),
            *[
                pl.col(f"wtc_ScYawPos_mean;{wtg}").radians().sin().alias(f"wtc_ScYawPos_mean_sin;{wtg}")
                for wtg in ref_wtgs
            ],
            *[
                pl.col(f"wtc_ScYawPos_mean;{wtg}").radians().cos().alias(f"wtc_ScYawPos_mean_cos;{wtg}")
                for wtg in ref_wtgs
            ],
            pl.concat_list([pl.col(f"wtc_AmbieTmp_mean;{wtg}") for wtg in ref_wtgs])
            .list.mean()
            .alias("ambient_temp_mean"),
        )
        .collect()
    )
    return (
        X.lazy()
        .with_columns(
            *[
                pl.col(col).shift(1).alias(col + "_lag10min")
                for col in X.columns
                if col not in ["TimeStamp_StartFormat", "is_valid", "seconds_since_2016"]
            ]
        )
        .with_columns(
            pl.col("TimeStamp_StartFormat").dt.minute().mul(2 * math.pi / 60).sin().alias("minutes_sin"),
            pl.col("TimeStamp_StartFormat").dt.minute().mul(2 * math.pi / 60).cos().alias("minutes_cos"),
            pl.col("TimeStamp_StartFormat").dt.hour().mul(2 * math.pi / 24).sin().alias("hours_sin"),
            pl.col("TimeStamp_StartFormat").dt.hour().mul(2 * math.pi / 24).cos().alias("hours_cos"),
            pl.col("TimeStamp_StartFormat").dt.ordinal_day().mul(2 * math.pi / 365).sin().alias("days_sin"),
            pl.col("TimeStamp_StartFormat").dt.ordinal_day().mul(2 * math.pi / 365).cos().alias("days_cos"),
            pl.col("TimeStamp_StartFormat").dt.month().mul(2 * math.pi / 12).sin().alias("months_sin"),
            pl.col("TimeStamp_StartFormat").dt.month().mul(2 * math.pi / 12).cos().alias("months_cos"),
        )
        .collect()
        .with_columns(
            pl.col("TimeStamp_StartFormat")
            .map_elements(lambda ts: sun_position.altitude(timestamp_utc=ts), return_dtype=pl.Float64)
            .mul(180 / math.pi)
            .alias("sun_altitude"),
        )
    )


def filter_is_valid(X: pl.DataFrame, y: pl.Series) -> tuple[pl.DataFrame, pl.Series]:
    y = y.filter(X.select("is_valid").to_series())
    X = X.filter(pl.col("is_valid"))
    return X, y


def select_features(X: pl.DataFrame, *, ref_wtgs: list[int]) -> pl.DataFrame:
    cols = [
        *[pl.col(f"wtc_AcWindSp_mean;{ref_wtg}") for ref_wtg in ref_wtgs],
        *[pl.col(f"wtc_AcWindSp_stddev;{ref_wtg}") for ref_wtg in ref_wtgs],
        *[pl.col(f"wtc_AcWindSp_min;{ref_wtg}") for ref_wtg in ref_wtgs],
        *[pl.col(f"wtc_AcWindSp_max;{ref_wtg}") for ref_wtg in ref_wtgs],
        *[pl.col(f"wtc_ScYawPos_mean_sin;{ref_wtg}") for ref_wtg in ref_wtgs],
        *[pl.col(f"wtc_ScYawPos_mean_cos;{ref_wtg}") for ref_wtg in ref_wtgs],
        *[pl.col(f"wtc_ScYawPos_stddev;{ref_wtg}") for ref_wtg in ref_wtgs],
        *[pl.col(f"wtc_ScReToOp_timeon;{ref_wtg}") for ref_wtg in ref_wtgs],
        *[pl.col(f"wtc_ActPower_mean;{ref_wtg}") for ref_wtg in ref_wtgs],
        *[pl.col(f"wtc_ActPower_stddev;{ref_wtg}") for ref_wtg in ref_wtgs],
        *[pl.col(f"wtc_ActPower_min;{ref_wtg}") for ref_wtg in ref_wtgs],
        *[pl.col(f"wtc_ActPower_max;{ref_wtg}") for ref_wtg in ref_wtgs],
        *[pl.col(f"wtc_GenRpm_mean;{ref_wtg}") for ref_wtg in ref_wtgs],
        *[pl.col(f"wtc_PitcPosA_mean;{ref_wtg}") for ref_wtg in ref_wtgs],
        *[pl.col(f"wtc_AcWindSp_mean;{ref_wtg}_lag10min") for ref_wtg in ref_wtgs],
        *[pl.col(f"wtc_ActPower_mean;{ref_wtg}_lag10min") for ref_wtg in ref_wtgs],
        pl.col("ambient_temp_mean"),
        pl.col("sun_altitude"),
        pl.col("seconds_since_2016"),
        pl.col("hours_sin"),
        pl.col("hours_cos"),
        pl.col("days_sin"),
        pl.col("days_cos"),
        pl.col("months_sin"),
        pl.col("months_cos"),
    ]
    return X.select(*cols)


def plot_generalization(
    automl: AutoML,
    *,
    X_train: pl.DataFrame,
    y_train: pl.Series,
    X_validation: pl.DataFrame,
    y_validation: pl.Series,
    variable_name: str,
    unit: str,
) -> None:
    train_prediction = pl.Series(values=automl.predict(X_train.to_pandas()))
    validation_prediction = pl.Series(values=automl.predict(X_validation.to_pandas()))
    mae_train = abs(y_train - train_prediction).mean()
    mae_validation = abs(y_validation - validation_prediction).mean()

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

    ax1.scatter(
        x=y_train.to_numpy().flatten(),
        y=train_prediction.to_numpy().flatten(),
        alpha=0.1,
    )
    ax1.text(0.05, 0.95, f"MAE: {mae_train:.2f} {unit}", ha="left", va="top", transform=ax1.transAxes)
    ax1.set_xlabel(f"True {variable_name} [{unit}]")
    ax1.set_ylabel(f"Predicted {variable_name} [{unit}]")
    ax1.set_title("Training set")
    ax1.grid(visible=True)

    ax2.scatter(
        x=y_validation.to_numpy().flatten(),
        y=validation_prediction.to_numpy().flatten(),
        alpha=0.1,
    )
    ax2.text(0.05, 0.95, f"MAE: {mae_validation:.2f} {unit}", ha="left", va="top", transform=ax2.transAxes)
    ax2.set_xlabel(f"True Wind {variable_name} [{unit}]")
    ax2.set_ylabel(f"Predicted {variable_name} [{unit}]")
    ax2.set_title("Validation set")
    ax2.grid(visible=True)


def plot_feature_importance(automl: AutoML) -> None:
    feature_importance = pl.DataFrame(
        {
            "Name": automl.feature_names_in_,
            "Importance": automl.feature_importances_,
        },
    ).sort("Importance")

    fig, ax = plt.subplots(figsize=(10, 0.17 * len(feature_importance)))
    bars = ax.barh(
        y=feature_importance.select("Name").to_numpy().flatten(),
        width=feature_importance.select("Importance").to_numpy().flatten(),
        alpha=0.7,
    )
    ax.bar_label(bars)
    ax.set_xlabel("Importance")
    ax.set_ylabel("Feature")
    ax.set_ylim(0.1, len(feature_importance))
    fig.tight_layout()

#### Model the wind speed

Wind speed is a key driver of active power output of a turbine. Therefore, we begin by modelling the wind speed of the target turbine, using only the features listed above.

The challenge aims to predict a year of data using historical data. Therefore, we take the last year of the training set as a validation set, and train the model in the remaining three years to start with. We use `XGBoost` since it can handle complex data with large numbers of features reasonably well. Hyperparameters are tuned using cross-fold validation and uniform splitting to maximise generalisation performance. Later, during retraining, we use time-aware splitting.

In [ ]:
X_train_ws = df_train.select(pl.exclude("wtc_AcWindSp_mean;1"))
y_train_ws = df_train.select("wtc_AcWindSp_mean;1").to_series()

X_test = load_submission_dataset().collect()

wf_lat_lon = load_turbine_metadata().select(pl.col("Latitude").mean(), pl.col("Longitude").mean()).collect()

training_mask = X_train_ws.select(
    pl.col("TimeStamp_StartFormat").lt(dt.datetime(2019, 1, 1, tzinfo=dt.UTC))
).to_series()
X_validation_ws = X_train_ws.filter(~training_mask)
y_validation_ws = y_train_ws.filter(~training_mask)
X_train_ws = X_train_ws.filter(training_mask)
y_train_ws = y_train_ws.filter(training_mask)

X_train_ws = preprocess(
    X_train_ws,
    ref_wtgs=[2, 3, 4, 5, 7],
    lat=wf_lat_lon.select("Latitude").item(),
    lon=wf_lat_lon.select("Longitude").item(),
)

X_train_ws, y_train_ws = filter_is_valid(X_train_ws, y_train_ws)
X_train_ws = select_features(X_train_ws, ref_wtgs=[2, 3, 4, 5, 7])

X_validation_ws = preprocess(
    X_validation_ws,
    ref_wtgs=[2, 3, 4, 5, 7],
    lat=wf_lat_lon.select("Latitude").item(),
    lon=wf_lat_lon.select("Longitude").item(),
)

X_validation_ws, y_validation_ws = filter_is_valid(X_validation_ws, y_validation_ws)
X_validation_ws = select_features(X_validation_ws, ref_wtgs=[2, 3, 4, 5, 7])

automl_ws = AutoML()
automl_settings = {
    "time_budget": time_budget,
    "task": "regression",
    "metric": "mae",
    "estimator_list": [
        "xgboost",
    ],
    "log_file_name": "automl.log",
    "seed": 42,
    "eval_method": "cv",
    "n_splits": 5,
    "split_type": "uniform",
    "early_stop": True,
}
automl_ws.fit(
    X_train=X_train_ws.to_pandas(),
    y_train=y_train_ws.to_pandas(),
    **automl_settings,
)

##### Assess generalisation

The MAEs obtained for wind speed are similar to the best entries in the [Kaggle competition to predict wind speed](https://www.kaggle.com/competitions/predict-the-wind-speed-at-a-wind-turbine). The model appears to generalise well; the scores for training and validation sets are similar.

In [ ]:
plot_generalization(
    automl_ws,
    X_train=X_train_ws,
    y_train=y_train_ws,
    X_validation=X_validation_ws,
    y_validation=y_validation_ws,
    variable_name="Wind Speed",
    unit="m/s",
)

##### Plot feature importance

As expected, wind speed and active power signals tend to dominate the feature importance. Yaw position and time in operation, while small, are crucial for modelling wake effects.

In [ ]:
plot_feature_importance(automl_ws)

##### Refit to the full training dataset

Finally, we refit the wind speed model to the full training dataset, again using cross-fold validation to ensure generalisation performance, but now using time-aware splitting to ensure that the model is trained on data that is always earlier than the validation set. The hyperparameters from the previous model are used as the startging point for the refitting.

In [ ]:
automl_ws_full = AutoML()
automl_settings = {
    "time_budget": time_budget,
    "task": "regression",
    "metric": "mae",
    "estimator_list": [
        "xgboost",
    ],
    "log_file_name": "automl.log",
    "seed": 42,
    "eval_method": "cv",
    "n_splits": 5,
    "split_type": "time",
    "early_stop": True,
    "starting_points": automl_ws.best_config_per_estimator,
}
automl_ws_full.fit(
    X_train=pl.concat([X_train_ws, X_validation_ws]).to_pandas(),
    y_train=pl.concat([y_train_ws, y_validation_ws]).to_pandas(),
    **automl_settings,
)

In [ ]:
train_prediction = pl.Series(values=automl_ws_full.predict(pl.concat([X_train_ws, X_validation_ws]).to_pandas()))
mae_train = abs(pl.concat([y_train_ws, y_validation_ws]) - train_prediction).mean()

fig, ax1 = plt.subplots(1, 1, figsize=(6, 5))

ax1.scatter(
    x=pl.concat([y_train_ws, y_validation_ws]).to_numpy().flatten(),
    y=train_prediction.to_numpy().flatten(),
    alpha=0.1,
)
ax1.text(0.05, 0.95, f"MAE: {mae_train:.2f}", ha="left", va="top", transform=ax1.transAxes)
ax1.set_xlabel("True")
ax1.set_ylabel("Predicted")
ax1.set_title("Full Train set")
ax1.grid(visible=True)

#### Predict the power

Finally, we repeat essentially the same process for the active power signal, adding in the modelled wind speed as an additional feature. Again, we first split the training set into the same training and validation sets, assess the generalisation performance, plot the feature importance, and then refit to the full training dataset using time-aware splitting.

In [ ]:
X_train = df_train.select(pl.exclude("target"))
y_train = df_train.select("target").to_series()

X_test = load_submission_dataset().collect()

wf_lat_lon = load_turbine_metadata().select(pl.col("Latitude").mean(), pl.col("Longitude").mean()).collect()

training_mask = X_train.select(pl.col("TimeStamp_StartFormat").lt(dt.datetime(2019, 1, 1, tzinfo=dt.UTC))).to_series()
X_validation = X_train.filter(~training_mask)
y_validation = y_train.filter(~training_mask)
X_train = X_train.filter(training_mask)
y_train = y_train.filter(training_mask)

X_train = preprocess(
    X_train,
    ref_wtgs=[2, 3, 4, 5, 7],
    lat=wf_lat_lon.select("Latitude").item(),
    lon=wf_lat_lon.select("Longitude").item(),
)

X_train, y_train = filter_is_valid(X_train, y_train)
X_train = select_features(X_train, ref_wtgs=[2, 3, 4, 5, 7])

assert len([col for col in X_train.columns if col.endswith(";1")]) == 0, (
    "Test turbine features should not be in training set"
)

X_validation = preprocess(
    X_validation,
    ref_wtgs=[2, 3, 4, 5, 7],
    lat=wf_lat_lon.select("Latitude").item(),
    lon=wf_lat_lon.select("Longitude").item(),
)

X_validation, y_validation = filter_is_valid(X_validation, y_validation)
X_validation = select_features(X_validation, ref_wtgs=[2, 3, 4, 5, 7])

X_train = X_train.with_columns(engineered_wind_speed=pl.Series(values=automl_ws_full.predict(X_train.to_pandas())))
X_validation = X_validation.with_columns(
    engineered_wind_speed=pl.Series(values=automl_ws_full.predict(X_validation.to_pandas()))
)

automl = AutoML()
automl_settings = {
    "time_budget": time_budget,
    "task": "regression",
    "metric": "mae",
    "estimator_list": [
        "xgboost",
    ],
    "log_file_name": "automl.log",
    "seed": 42,
    "eval_method": "cv",
    "n_splits": 5,
    "split_type": "uniform",
    "early_stop": True,
}
automl.fit(
    X_train=X_train.to_pandas(),
    y_train=y_train.to_pandas(),
    **automl_settings,
)

##### Assess generalisation

The model overfits slightly; in particular there are a few outliers at low power values (100-300kW) that need investigating. But overall the generalisation is reasonable.

In [ ]:
plot_generalization(
    automl,
    X_train=X_train,
    y_train=y_train,
    X_validation=X_validation,
    y_validation=y_validation,
    variable_name="Active Power",
    unit="kW",
)

##### Plot feature importance

As expected the engineered wind speed feature is the most important, followed by active power and wind speed signals from the reference turbines.

In [ ]:
plot_feature_importance(automl)

##### Refit to the full training dataset

This follows the same process as for the wind speed model.

In [ ]:
automl_full = AutoML()
automl_settings = {
    "time_budget": time_budget,
    "task": "regression",
    "metric": "mae",
    "estimator_list": [
        "xgboost",
    ],
    "log_file_name": "automl.log",
    "seed": 42,
    "eval_method": "cv",
    "n_splits": 5,
    "split_type": "time",
    "early_stop": True,
    "starting_points": automl.best_config_per_estimator,
}
automl_full.fit(
    X_train=pl.concat([X_train, X_validation]).to_pandas(),
    y_train=pl.concat([y_train, y_validation]).to_pandas(),
    **automl_settings,
)

In [ ]:
train_prediction = pl.Series(values=automl_full.predict(pl.concat([X_train, X_validation]).to_pandas()))
mae_train = abs(pl.concat([y_train, y_validation]) - train_prediction).mean()

fig, ax1 = plt.subplots(1, 1, figsize=(6, 5))

ax1.scatter(
    x=pl.concat([y_train, y_validation]).to_numpy().flatten(),
    y=train_prediction.to_numpy().flatten(),
    alpha=0.1,
)
ax1.text(0.05, 0.95, f"MAE: {mae_train:.2f} kW", ha="left", va="top", transform=ax1.transAxes)
ax1.set_xlabel("True Active Power [kW]")
ax1.set_ylabel("Predicted Active Power [kW]")
ax1.set_title("Full Train set")
ax1.grid(visible=True)

### Generate predictions for submission

In [ ]:
X_test = load_submission_dataset().collect()
df_id = X_test.select("id")

X_test = preprocess(
    X_test,
    ref_wtgs=[2, 3, 4, 5, 7],
    lat=wf_lat_lon.select("Latitude").item(),
    lon=wf_lat_lon.select("Longitude").item(),
)

X_test = select_features(X_test, ref_wtgs=[2, 3, 4, 5, 7])

X_test = X_test.with_columns(engineered_wind_speed=pl.Series(values=automl_ws_full.predict(X_test.to_pandas())))
y_test = pl.Series(values=automl_full.predict(X_test.to_pandas()))

submission = df_id.with_columns(prediction=y_test)

### Check and save the submission file

In [ ]:
# checking the columns are the expected ones
assert submission.columns == ["id", "prediction"], f'Expected columns ["id", "prediction"], found: {submission.columns}'

# checking no nulls in the data
assert submission.select(pl.col("id").is_null().sum()).item() == 0, "There are null values in the 'id' column"
assert submission.select(pl.col("id").is_nan().sum()).item() == 0, "There are nan values in the 'id' column"
assert submission.select(pl.col("prediction").is_null().sum()).item() == 0, (
    "There are null values in the 'prediction' column"
)
assert submission.select(pl.col("prediction").is_nan().sum()).item() == 0, (
    "There are nan values in the 'prediction' column"
)

# checking the row ids are unique and within expected range
duplicated_ids = submission.select("id").is_duplicated()
assert not duplicated_ids.any(), (
    f"There are duplicated ids: {submission.select('id').filter(duplicated_ids).to_series().unique()}"
)
invalid_ids = set(submission.select("id").unique().to_series().to_list()) - set(range(52704))
assert not invalid_ids, f"The following row IDs are not within the expected ones: {invalid_ids}"

print("Submission file is valid and ready for submission.")

submission.write_csv("submission.csv")